# Inicialização

In [1]:
# Se estiver rodando em um Colab, descomente a linha abaixo
# !pip install snscrape tqdm

In [2]:
from dataclasses import fields
from datetime import datetime
import pandas as pd
import snscrape.modules.twitter as sntwitter
from tqdm import tqdm

# Oculta warnings do notebook
# import warnings
# warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 80)
pd.options.display.float_format = '{:,.2f}'.format

# Contagem de tweets por candidato

In [3]:
# Contas dos candidatos eleitos
users = [
    'dreltonjr', 'EdianeMariaMTST', 'fabibbolsonaro', 'delegadoolim', 'leosiqueirabr', 'bethsahao', 
    'sppretas', 'tomeabduch', 'prcarloscezar', 'majormecca', 'valbolsonaro', 'zimbaldirafa', 
    'CarlosGiannazi', 'tenente_coimbra', 'rafaelsaraivasp', 'BrunoZambelli3', 'LFTeixeira13', 
    'EmidioDeSouza_', 'esuplicy', 'depeniotatto', 'eniotatto13114', 'anacarolserra23',
    'xerifedoconsum', 'auricchiothiago', 'itamar_borges', 'simaopedro_SP', 'DaniloBalas', 'vcamarinha', 
    'marcosdamasiosp', 'lcmarcolino', 'danialonsoDA', 'bfeministapsol', 'gersonpessoa',
    'deleg_graciela', 'ValdomiroLopes_', 'lucasbovesp', 'DANIELBRSOARES', 'paulomansur_', 
    'andreawerner_', 'CONTELOPES', 'anaperugini', 'professorabebel', 'EdmirChedid', 'deputadomaurici',
    'carteiroreaca', 'caiofranca40', 'marcialiapt13', 'letsaguiar', 'thainarafariapt', 'depbrunafurlan'
]

since = '2022-09-01'
until = '2022-11-01'

In [4]:
user_tweets = {}

for i, user in enumerate(users):
    query = f'from:{user} since:{since} until:{until}'
    user_scrapping_results = sntwitter.TwitterSearchScraper(query)
    tweet_ids = []
    for tweet in user_scrapping_results.get_items():
        tweet_ids.append(tweet.id)
    user_tweets[user] = tweet_ids
    print(f'{i}/{len(users)} {user}: {len(tweet_ids)} tweets')

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=from%3Adreltonjr+since%3A2022-09-01+until%3A2022-11-01&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&include_ext_edit_c

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=from%3Adreltonjr+since%3A2022-09-01+until%3A2022-11-01&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&inc

# Scrapping de tweets por filtro

In [ ]:
# Começamos filtrando todos os tweets de um usuario, dentro do período desejado
# Para buscar outros tweets, é só alterar estes parâmetros
username = 'ErikakHilton'
since = '2022-10-01'
until = '2022-11-01'

In [ ]:
# Montando e executando a busca
query = f'from:{username} since:{since} until:{until}'
user_scrapping_results = sntwitter.TwitterSearchScraper(query)

# Listando o id de todos os tweets retornados
tweet_ids = []
for tweet in tqdm(user_scrapping_results.get_items()):
    tweet_ids.append(tweet.id)

print(len(tweet_ids))

In [ ]:
# Vamos iterar entre os id's para pegar os dados dos tweets e também suas respostas
# Esta etapa pode demorar um pouco
tweets_and_replies = []

for tweet_id in tqdm(tweet_ids):
    tweet_scrapper = sntwitter.TwitterTweetScraper(tweet_id, mode=sntwitter.TwitterTweetScraperMode.SCROLL)
    for tweet in tweet_scrapper.get_items():
        tweets_and_replies.append(tweet)
        
print(len(tweets_and_replies))

In [ ]:
# Convertendo as informações para DataFrame
tweet_fields = [f.name for f in fields(tweet)]
user_fields = [f.name for f in fields(tweet.user)]
all_columns = tweet_fields + ['user_' + f for f in user_fields]

tweets_data = [
    [getattr(t, field) for field in tweet_fields] + 
    [getattr(t.user, field) for field in user_fields] 
    for t in tweets_and_replies
]
tweets_df = pd.DataFrame(tweets_data, columns=all_columns)

print(tweets_df.shape)
tweets_df.head()

In [ ]:
tweets_df.info()

In [ ]:
tweets_df.iloc[0]

In [ ]:
# Exportando dados scrapeados
# Para salvar os dados é só descomentar as linhas abaixo

# output_path = 'datasets/scrapped/'
# output_file = f'{username}-{since}-{until}-{datetime.now().isoformat()}.csv'

# tweets_df.to_csv(f"{output_path}{output_file}", sep=';', encoding='utf-8', index=False)

In [5]:
username = 'ErikakHilton'

tweet_obj = next(sntwitter.TwitterProfileScraper(username).get_items())
last_tweet

Tweet(url='https://twitter.com/ErikakHilton/status/1576714415231336448', date=datetime.datetime(2022, 10, 2, 23, 23, 5, tzinfo=datetime.timezone.utc), rawContent='Já dá pra dizer: TRAVESTI PRETA ELEITA! https://t.co/U7nkJwyGMM', renderedContent='Já dá pra dizer: TRAVESTI PRETA ELEITA! https://t.co/U7nkJwyGMM', id=1576714415231336448, user=User(username='ErikakHilton', id=738143559920934912, displayname='ERIKA HILTON', rawDescription='Deputada Federal - PSOL/SP (2023-2027)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.', renderedDescription='Deputada Federal - PSOL/SP (2023-2027)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.', descriptionLinks=None, verified=False, created=datetime.datetime(2016, 6, 1, 23, 1, 53, tzinfo=datetime.timezone.utc), followersCount=266261, friendsCount=4805, statusesCount=27219, favouritesCount=66036, listedCount=480, mediaCount=3443, locati

In [ ]:
limit = datetime.datetime(2023, 4, 1, tzinfo=datetime.timezone.utc)

i = 0
tweets = []
twitter_generator = sntwitter.TwitterProfileScraper(username).get_items()

for tweet in twitter_generator:
    print(tweet.url, tweet.date)
    if tweet.date < limit and len(tweets) > 5:
    #if i > 10:
        print('out')
        break
    
    tweets.append(tweet)
    i += 1

In [ ]:
tweet

In [ ]:
Tweet(
    url="https://twitter.com/ErikakHilton/status/1576714415231336448",
    date=datetime.datetime(2022, 10, 2, 23, 23, 5, tzinfo=datetime.timezone.utc),
    rawContent="Já dá pra dizer: TRAVESTI PRETA ELEITA! https://t.co/U7nkJwyGMM",
    renderedContent="Já dá pra dizer: TRAVESTI PRETA ELEITA! https://t.co/U7nkJwyGMM",
    id=1576714415231336448,
    user=User(
        username="ErikakHilton",
        id=738143559920934912,
        displayname="ERIKA HILTON",
        rawDescription="Deputada Federal - PSOL/SP (2023-2027)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.",
        renderedDescription="Deputada Federal - PSOL/SP (2023-2027)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.",
        descriptionLinks=None,
        verified=False,
        created=datetime.datetime(2016, 6, 1, 23, 1, 53, tzinfo=datetime.timezone.utc),
        followersCount=266261,
        friendsCount=4805,
        statusesCount=27219,
        favouritesCount=66036,
        listedCount=480,
        mediaCount=3443,
        location="São Paulo, Brasil",
        protected=False,
        link=TextLink(
            text="erikahilton.com.br",
            url="http://erikahilton.com.br/",
            tcourl="https://t.co/5KKZ3fVaUj",
            indices=(0, 23),
        ),
        profileImageUrl="https://pbs.twimg.com/profile_images/1559380990333706240/vEImKRHW_normal.jpg",
        profileBannerUrl="https://pbs.twimg.com/profile_banners/738143559920934912/1670375803",
        label=None,
    ),
    replyCount=2678,
    retweetCount=15682,
    likeCount=93668,
    quoteCount=3398,
    conversationId=1576714415231336448,
    lang="pt",
    source=None,
    sourceUrl=None,
    sourceLabel=None,
    links=None,
    media=[
        Photo(
            previewUrl="https://pbs.twimg.com/media/FeGdZ39X0AEsYnC?format=jpg&name=small",
            fullUrl="https://pbs.twimg.com/media/FeGdZ39X0AEsYnC?format=jpg&name=orig",
            altText=None,
        )
    ],
    retweetedTweet=None,
    quotedTweet=None,
    inReplyToTweetId=None,
    inReplyToUser=None,
    mentionedUsers=None,
    coordinates=None,
    place=None,
    hashtags=None,
    cashtags=None,
    card=None,
    viewCount=None,
    vibe=None,
)
